In [2]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import string
import os
from tqdm import tqdm

In [2]:
# define lyrics scraping;
# from https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3

#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    #print(html)
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find_all("div", class_="Lyrics__Container-sc-1ynbvzw-6 jYfhrf")
    if lyrics1:
        lyrics = lyrics1.get_text("\m")
    elif lyrics2:
        #print(lyrics2)
        lyrics = ""
        for l in lyrics2: 
            lyrics += l.get_text("\m")
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['track']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1

In [5]:
#client credentials id 

cid = '3a13f1e179e64b1194a92ca3a81fe54f'
secret = '883a5a29ea7e442aa3dd47ee2f74a9d7'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
songs = pd.read_csv('deezer_mood_detection_dataset/train.csv', delimiter = ',')
n_songs = 50

In [6]:
#first 50 train songs 

tester = songs.head(n_songs)
# tester

In [7]:
#dataframe with track names and artists

df = pd.concat([tester['track_name'], tester['artist_name']], axis = 1, keys = ['track_name', 'artist_name'])


In [8]:
df

,track_name,artist_name
0,Insanity's Crescendo,Dark Tranquillity
1,Stabat mater dolorosa,Anorexia Nervosa
2,Ain't Nobody's Business,Jimmy Witherspoon
3,Out Of Touch,Uniting Nations
4,Brandy alexander,The Walkmen
5,Caravan,Duke Ellington
6,Tea For Two,Art Tatum
7,Lullaby Of Birdland,Ella Fitzgerald
8,Pop Song,cLOUDDEAD
9,Son Of A Gun,cLOUDDEAD


In [20]:
# features dictionary 

data_dict = {'danceability': [],
             'energy':[],
             'key': [],
             'loudness': [],
             'mode': [],
             'speechiness': [],
             'acousticness': [],
             'instrumentalness': [],
             'liveness': [],
             'valence': [],
             'tempo': [],
             'id': [],
             'time_signature': [], 
             'lyrics': [],
            'artist':[],
            'trackname': []}

    

for i in range(n_songs):
    artist_name = df.iloc[i][1]
    track_name = df.iloc[i][0]
    
    #search for spotify tack 
    track_results = sp.search(q=f'artist: {artist_name}, track:{track_name}', type='track', limit=10,offset=0)
    if len(track_results['tracks']['items']) > 0: #if track exists
        if track_results['tracks']['items'][0]['name'] == track_name: #validating track name 
            
            #get audio features and append to feature dictionary 
            ID = track_results['tracks']['items'][0]['id']
            audio_features = sp.audio_features(ID)[0] 
            for key in list(data_dict.keys())[:-3]:#untill -3 because the last 3 features are lyrics artist, trackname a
                data_dict[key].append(audio_features[key])
                
            #scrape lyrics from genius.com
            try:
                track_name_process = re.sub(r'[^\w\s]', '', track_name)   
                lyrics = scrape_lyrics(artist_name, track_name_process)
                data_dict['lyrics'].append(lyrics)
#                 print(lyrics)
            
            #if not found append None
            except:  
                print(track_name)
                data_dict['lyrics'].append(None)
        
                
#             attempt to addd artist and trackname, but i think somehting is still wrong
            data_dict['artist'].append(artist_name)
            data_dict['trackname'].append(track_name)

    

    if i %5 ==0: #every 100 songs, save the dateframe to a csv
        df_features = pd.DataFrame(data_dict) #convert dict to df
        df_features.to_csv('features.csv')
    
 

  
        

  2%|▏         | 1/50 [00:01<01:37,  1.99s/it]


Lullaby Of Birdland


AttributeError: 'tqdm' object has no attribute 'sleep'

In [39]:
# dataset = 'train'
# file = dataset + '.csv'
# path = os.path .join('deezer_mood_detection_dataset',file)
# songs = pd.read_csv(path, delimiter = ',')
# songs.head()

In [26]:
def features_to_csv(dataset, n_songs = 0, whole_dataset = False):
    """function that gets all the audio features from spotify and the lyrics from genius and saves it as a csv
    dataset: string value that is either 'train', 'test' or 'validation' to indicate which dataset to use
    n_songs = int value, depening on how many songs you want to get the track features and lyrics from
    whole_dataset: bool, if true it will get the whole dataset, otherwise, the specified n_songs
    it returns df_features"""
    # features dictionary 


    file = dataset + '.csv'
    path = os.path.join('deezer_mood_detection_dataset',file)
    songs = pd.read_csv(path, delimiter = ',')
    
#     tester = songs.head(n_songs)
#     df = pd.concat([tester['track_name'], tester['artist_name']], axis = 1, keys = ['track_name', 'artist_name'])

    if whole_dataset == False:
        data = songs.head(n_songs)
    else:
        data = songs
        n_songs = len(data)

    df = pd.concat([data['track_name'], data['artist_name']], axis = 1, keys = ['track_name', 'artist_name'])
    
    data_dict = {'danceability': [],
                 'energy':[],
                 'key': [],
                 'loudness': [],
                 'mode': [],
                 'speechiness': [],
                 'acousticness': [],
                 'instrumentalness': [],
                 'liveness': [],
                 'valence': [],
                 'tempo': [],
                 'id': [],
                 'time_signature': [], 
                 'lyrics': [],
                'artist':[],
                'trackname': []}

    data_dict_copy = data_dict.copy()

    pbar = tqdm(total=n_songs)

    headers = pd.DataFrame(data_dict)
    headers.to_csv('features_'+ dataset +'.csv')


    for i in range(n_songs):
        artist_name = df.iloc[i][1]
        track_name = df.iloc[i][0]

        #search for spotify tack 
        track_results = sp.search(q=f'artist: {artist_name}, track:{track_name}', type='track', limit=10,offset=0)
        if len(track_results['tracks']['items']) > 0: #if track exists
            if track_results['tracks']['items'][0]['name'] == track_name: #validating track name 

                #get audio features and append to feature dictionary 
                ID = track_results['tracks']['items'][0]['id']
                audio_features = sp.audio_features(ID)[0] 
                for key in list(data_dict_copy.keys())[:-3]:#untill -3 because the last 3 features are lyrics artist, trackname a
                    data_dict_copy[key].append(audio_features[key])

                #scrape lyrics from genius.com
                try:
                    track_name_process = re.sub(r'[^\w\s]', '', track_name)   
                    lyrics = scrape_lyrics(artist_name, track_name_process)
                    data_dict_copy['lyrics'].append(lyrics)
    #                 print(lyrics)
                #if not found append None
                except:  
                    #print(track_name)
                    data_dict_copy['lyrics'].append(None)


    #             attempt to addd artist and trackname, but i think somehting is still wrong
                data_dict_copy['artist'].append(artist_name)
                data_dict_copy['trackname'].append(track_name)

        pbar.update(1) 
        
        # if i %10 ==0: #every 100 songs, save the dateframe to a csv
        #     df_features = pd.DataFrame(data_dict_copy) #convert dict to df
        #     df_features.to_csv('features_'+ dataset +'.csv', mode='a', header=False)
        #     data_dict_copy = data_dict.copy()
        #     #print(f'df saved at the {i}th iteration')

    pbar.close()

#     save df at the end
    df_features = pd.DataFrame(data_dict_copy) #convert dict to df
    df_features.to_csv('features_'+ dataset +'.csv', mode='a', header=False)
    return 

    

# run this function with both 'train', 'test' and 'validation'

In [28]:
df100 = features_to_csv(dataset = 'validation', n_songs = 0, whole_dataset = True)


























































  1%|          | 30/3863 [00:57<2:03:24,  1.93s/it]





































































































KeyboardInterrupt: 

In [21]:
df150 = pd.read_csv('features_validation.csv')
df150['lyrics'].isna().sum()
len(df150)

101

In [88]:
df100.to_csv('validation_partly.csv', sep = ';')


In [ ]:
# # features dictionary 
# def features_to_csv(data, n_songs)
#     data_dict = {'danceability': [],
#                  'energy':[],
#                  'key': [],
#                  'loudness': [],
#                  'mode': [],
#                  'speechiness': [],
#                  'acousticness': [],
#                  'instrumentalness': [],
#                  'liveness': [],
#                  'valence': [],
#                  'tempo': [],
#                  'id': [],
#                  'time_signature': [], 
#                  'lyrics': []}


#     for i in range(n_songs):
#         artist_name = df.iloc[i][1]
#         track_name = df.iloc[i][0]

#         #search for spotify tack 
#         track_results = sp.search(q=f'artist: {artist_name}, track:{track_name}', type='track', limit=10,offset=0)
#         if len(track_results['tracks']['items']) > 0: #if track exists
#             if track_results['tracks']['items'][0]['name'] == track_name: #validating track name 

#                 #get audio features and append to feature dictionary 
#                 ID = track_results['tracks']['items'][0]['id']
#                 audio_features = sp.audio_features(ID)[0] 
#                 for key in list(data_dict.keys())[:-1]:
#                     data_dict[key].append(audio_features[key])

#                 #scrape lyrics from genius.com
#                 try:
#                     track_name_process = re.sub(r'[^\w\s]', '', track_name)   
#                     lyrics = scrape_lyrics(artist_name, track_name_process)
#                     data_dict['lyrics'].append(lyrics)

#                 #if not found append None
#                 except:  
#                     print(track_name)
#                     data_dict['lyrics'].append(None)
#         if i %100 ==0: #every 100 songs, save the dateframe to a csv
#             df_features = pd.DataFrame(data_dict) #convert dict to df
#             df_features.to_csv('')

        

In [438]:
df = pd.DataFrame(data_dict)
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,lyrics
0,0.320,0.798,6,-9.660,0,0.0584,0.001520,0.251000,0.0734,0.4070,138.046,3SS3dDG9lKQxvfcJ7O4oTN,3,"Gently, hold our heads\nGently, hold our heads..."
1,0.434,0.600,5,-10.784,0,0.0286,0.867000,0.451000,0.1110,0.7530,106.511,3Jw4lBWbcznhlitliEMW0x,4,Night and stars above that shine so bright\nTh...
2,0.530,0.270,5,-11.251,0,0.0365,0.910000,0.000000,0.3120,0.5150,110.844,3RtJAYS51S5W6EMxrm9aMb,4,"Oh, lullaby by birdland that's what I\nAlways ..."
3,0.479,0.528,9,-10.104,0,0.1040,0.580000,0.001280,0.8140,0.1850,115.929,3ayXOJdCoW6noN3nJLXCxz,4,"[Intro]\nTwo, three\nI can't do it without-\nG..."
4,0.639,0.893,9,-6.509,1,0.0360,0.289000,0.008160,0.2220,0.9660,126.143,5jkFvD4UJrmdoezzT1FRoP,4,[Verse 1]\nThere lived a certain man in Russia...
5,0.462,0.602,7,-4.167,1,0.0285,0.142000,0.000000,0.1560,0.1840,75.071,2rVnxIk0QkCpHNWXElXtVk,4,[Verse 1]\nThere’s nothing I could say to you\...
6,0.638,0.656,5,-5.886,1,0.0357,0.188000,0.000000,0.1460,0.2250,104.036,7wZUrN8oemZfsEd1CGkbXE,4,"[Verse 1]\nClosed off from love, I didn't need..."
7,0.684,0.455,5,-8.216,1,0.0307,0.215000,0.000001,0.2630,0.4870,97.956,174rZBKJAqD10VBnOjlQQ3,4,[Verse 1]\nHow do I breathe?\nHow do I breathe...
8,0.540,0.206,7,-14.580,1,0.0298,0.907000,0.000001,0.1350,0.1020,109.297,4YtVg22zJpwYWgWFpj5qH6,4,If I could crawl into a shell\nThen I wouldn't...
9,0.418,0.578,9,-8.402,1,0.0287,0.753000,0.000001,0.2440,0.6400,137.359,52VnbCW97efS5ycFHiw15f,3,"[Chorus]\nI will never see the sun\nSpadina, S..."


In [27]:
#https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3

#function to scrape lyrics from genius
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    #print(html)
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find_all("div", class_="Lyrics__Container-sc-1ynbvzw-6 jYfhrf")
    if lyrics1:
        lyrics = lyrics1.get_text("\n")
    elif lyrics2:
        #print(lyrics2)
        lyrics = ""
        for l in lyrics2: 
            lyrics += l.get_text("\n")
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['track']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1